In [366]:
import pandas as pd
import numpy as np

# Load data, split into training and validation sets

In [367]:
filepath = './train.csv'
dataframe = pd.read_csv(filepath)
print('train set size:', len(dataframe))
# print(len(dataframe))

train set size: 7239


## Also load test data (no splitting needed)

In [368]:
test_filepath = './test.csv'
test_dataframe = pd.read_csv(test_filepath)
print('test set size:', len(test_dataframe))
# print(test_dataframe)

test set size: 2413


In [369]:
train_ratio = 0.7 # 70% for training, 30% for validation
random_seed = 290 # a fixed random seed allows fixed random runs (for controlled debugging). set to None to be random.

train_dataframe = dataframe.sample(frac= train_ratio, random_state=100) 
valid_dataframe = dataframe.drop(train_dataframe.index)

#train_dataframe = processed_dataframe.sample(frac= train_ratio, random_state=100) 
#valid_dataframe = processed_dataframe.drop(train_dataframe.index)
print('training set size:', len(train_dataframe))
print('validation set size:', len(valid_dataframe))
# print(train_dataframe)

training set size: 5067
validation set size: 2172


In [370]:
# take out prediction targets: mean grades 
train_Y = train_dataframe['meanGrade']
valid_Y = valid_dataframe['meanGrade']

In [371]:
train_dataframe.head()

,id,original,edit,grades,meanGrade
1200,7395,Congress OKs Trump bid to <widen/> private car...,destroy,20000,0.4
5865,4484,Trump and Obama have the same approval rating ...,person,21100,0.8
6423,13227,H.R. McMaster says Trump administration will c...,lizard,11100,0.6
3398,14345,Triple Threat : New <Pneumonia/> Is Drug-Resis...,President,22210,1.4
4629,10432,Is it <Watergate/> yet ?,moving,10000,0.2


In [372]:
train_dataframe.edit.nunique()

3356

In [373]:
# Generate edited title
edit_corpus = []
for index,row in train_dataframe.iterrows():
    title = row["original"]
    startpos = title.find("<")
    endpos = title.find("/>")
    edit_title = title[:startpos] + row.edit+title[endpos+2:]
    edit_corpus.append(edit_title)

In [374]:
original_corpus = []
for index,row in train_dataframe.iterrows():
    title = title.replace("<","").replace("/>","")
    original_corpus.append(edit_title)

In [375]:
original_corpus[2]

'Sen. Bob Hertzberg will cooperate with investigation into unwanted hairpieces '

In [376]:
import gensim
from gensim import utils
import gensim.downloader
import numpy as np
import sys
from sklearn.datasets import fetch_20newsgroups
from nltk import word_tokenize
from nltk import download
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
%matplotlib inline

In [377]:
model = gensim.downloader.load('word2vec-google-news-300')

# WordEmbedding

In [378]:
def preprocess(text):
    text = text.lower()
    doc = word_tokenize(text)
    doc = [word for word in doc if word not in stop_words]
    doc = [word for word in doc if word.isalpha()] #restricts string to alphabetic characters only
    return doc

In [379]:
#Filter corpus, texts given the function condition_on_doc which takes a doc.
def filter_test_docs(corpus, texts, condition_on_doc):
    number_of_docs = len(corpus)

    if texts is not None:
        texts = [text for (text, doc) in zip(texts, corpus)
                 if condition_on_doc(doc)]

    #labels = [i for (i, doc) in zip(labels, corpus) if condition_on_doc(doc)]
    corpus = [doc for doc in corpus if condition_on_doc(doc)]

    print("{} docs removed".format(number_of_docs - len(corpus)))

    return (corpus, texts)

In [380]:
def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.index_to_key]
    return np.mean(word2vec_model[doc], axis=0)

In [381]:
def has_vector_representation(word2vec_model, doc):
    """check if at least one word of the document is in the
    word2vec dictionary"""
    return not all(word not in word2vec_model.index_to_key for word in doc)

In [382]:
stop_words = stopwords.words('english')
corpus = [preprocess(text) for text in edit_corpus]

In [383]:
#train dataset preparation
train_edit_corpus = []
for index,row in train_dataframe.iterrows():
    title = row["original"]
    startpos = title.find("<")
    endpos = title.find("/>")
    edit_title = title[:startpos] + row.edit+title[endpos+2:]
    train_edit_corpus.append(edit_title)
TrainY = train_Y.tolist()
traincorpus = [preprocess(text) for text in train_edit_corpus]
traincorpus, train_edit_corpus = filter_test_docs(traincorpus, train_edit_corpus, lambda doc: (len(doc) != 0))
traincorpus, train_edit_corpus = filter_test_docs(traincorpus, train_edit_corpus, lambda doc: has_vector_representation(model, doc))

0 docs removed
0 docs removed


In [511]:
len(train_edit_corpus)

5067

In [384]:
trainx =[]
for doc in corpus: #look up each doc in model
    trainx.append(document_vector(model, doc))

In [385]:
train_X_wordfeature = np.array(trainx) #list to array
train_X_wordfeature.shape, len(TrainY)

((5067, 300), 5067)

In [386]:
train_X_wordfeature[1]

array([ 0.02250366, -0.05791016,  0.02400055,  0.12885742, -0.0472168 ,
       -0.07488861,  0.05567627, -0.08117676,  0.22072753, -0.01972656,
       -0.04659119, -0.12818603, -0.03125305,  0.03586426, -0.16018638,
        0.1545227 ,  0.00521545, -0.00149078, -0.02518311, -0.05304871,
        0.01820068,  0.01730042, -0.01219482, -0.06372681, -0.02220154,
       -0.03897247, -0.09849854,  0.07667847,  0.02389011,  0.01425781,
        0.03077011,  0.01958313, -0.12807007,  0.04484253, -0.01698608,
        0.03834839, -0.04782701,  0.00859375, -0.02968445,  0.0531311 ,
        0.07154846, -0.15134887,  0.07699585,  0.09118652,  0.04770508,
        0.00410156,  0.12024879, -0.00957642, -0.0388916 ,  0.02526245,
       -0.02249756,  0.03730469,  0.03029442, -0.04067535,  0.00602188,
        0.02783203, -0.11663818, -0.06808472,  0.050177  , -0.03319092,
       -0.04821777,  0.159729  , -0.07427216, -0.0378479 ,  0.046521  ,
        0.02170639, -0.04903259,  0.03076782, -0.04892578,  0.03

In [387]:
#valid dataset preparation
valid_edit_corpus = []
for index,row in valid_dataframe.iterrows():
    title = row["original"]
    startpos = title.find("<")
    endpos = title.find("/>")
    edit_title = title[:startpos] + row.edit+title[endpos+2:]
    valid_edit_corpus.append(edit_title)
ValidY=valid_Y.tolist()
validcorpus = [preprocess(text) for text in valid_edit_corpus]
validcorpus, valid_edit_corpus = filter_test_docs(validcorpus, valid_edit_corpus, lambda doc: (len(doc) != 0))
validcorpus, valid_edit_corpus = filter_test_docs(validcorpus, valid_edit_corpus, lambda doc: has_vector_representation(model, doc))

0 docs removed
0 docs removed


In [388]:
valid_x =[]
for doc in validcorpus: #look up each doc in model
    valid_x.append(document_vector(model, doc))

In [389]:
Valid_X_wordfeature = np.array(valid_x) #list to array
Valid_X_wordfeature.shape, len(ValidY)

((2172, 300), 2172)

In [390]:
#test_dataset wordembedding feature generation
test_edit_corpus = []
for index,row in test_dataframe.iterrows():
    title = row["original"]
    startpos = title.find("<")
    endpos = title.find("/>")
    edit_title = title[:startpos] + row.edit+title[endpos+2:]
    test_edit_corpus.append(edit_title)
    
testcorpus = [preprocess(text) for text in test_edit_corpus]
testcorpus, test_edit_corpus = filter_test_docs(testcorpus, test_edit_corpus, lambda doc: (len(doc) != 0))
testcorpus, test_edit_corpus = filter_test_docs(testcorpus, test_edit_corpus, lambda doc: has_vector_representation(model, doc))

0 docs removed
0 docs removed


In [24]:
len(testcorpus)

2413

In [503]:
test_x =[]
for doc in testcorpus: #look up each doc in model
    test_x.append(document_vector(model, doc))

In [504]:
test_X = np.array(test_x) #list to array
test_X.shape

(2413, 300)

# Topic Features

In [393]:
from gensim import corpora
import gensim
import nltk

In [394]:
# take out all title texts in a list
titles = []
for index, row in train_dataframe.iterrows():
#     print (index, row['original'], '|', row['edit'])
    title_text = row['original'].replace('<', '').replace('/>', '')
    titles.append( title_text )

In [395]:
en_stop = set(nltk.corpus.stopwords.words('english'))

In [396]:
# process a text string into a list of tokens

def prepare_text_for_lda(text):
    # convert all words into lower case, split by white space
    tokens = text.strip().lower().split()
    
    #  remove words with 1 or 2 letters (small words, punctuation)
    tokens = [token for token in tokens if len(token) > 2]
    
    # remove English stopwords (as defined by NLTK)
    tokens = [token for token in tokens if token not in en_stop]
    return tokens

# convert the corpus into a sparse matrix format for gensim

text_data = []
for title in titles:
    title = prepare_text_for_lda(title)
    text_data.append(title)
    
# print(text_data)
            
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

# print(corpus)

# train latent Dirichlet topic model

NUM_TOPICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)


topics = ldamodel.print_topics(num_words=6)
for topic in topics:
    print(topic)

(0, '0.047*"trump" + 0.024*"health" + 0.016*"twitter" + 0.013*"care" + 0.012*"like" + 0.011*"help"')
(1, '0.052*"trump" + 0.016*"court" + 0.015*"minister" + 0.013*"hits" + 0.012*"rally" + 0.012*"says"')
(2, '0.057*"trump" + 0.057*"russia" + 0.018*"says" + 0.014*"u.s." + 0.012*"report" + 0.011*"probe"')
(3, '0.024*"tax" + 0.019*"trump" + 0.015*"office" + 0.014*"year" + 0.012*"chemical" + 0.012*"india"')
(4, '0.062*"trump" + 0.020*"climate" + 0.015*"change" + 0.010*"check" + 0.009*"dies" + 0.009*"business"')
(5, '0.057*"korea" + 0.052*"north" + 0.036*"trump" + 0.015*"u.s." + 0.013*"says" + 0.012*"south"')
(6, '0.028*"trump" + 0.012*"state" + 0.012*"government" + 0.011*"syria" + 0.010*"new" + 0.009*"israel"')
(7, '0.081*"house" + 0.073*"white" + 0.054*"trump" + 0.015*"fbi" + 0.010*"press" + 0.009*"america"')
(8, '0.024*"deal" + 0.022*"shooting" + 0.021*"million" + 0.016*"trump" + 0.016*"senate" + 0.015*"manafort"')
(9, '0.046*"trump" + 0.022*"president" + 0.014*"michael" + 0.010*"cohen" +

In [397]:
#train dataset topic feature generation
topicvector = np.zeros((len(train_dataframe),20))

In [398]:
np.shape(topicvector)

(5067, 20)

In [399]:
trainrow=0
for index, row in train_dataframe.iterrows():
#     print (index, row['original'], '|', row['edit'])
    title_text = row['original'].replace('<', '').replace('/>', '')
    title_text = prepare_text_for_lda(title_text)
    title_doc_bow = dictionary.doc2bow(title_text)
    predicted_topics = ldamodel.get_document_topics(title_doc_bow)
    for k,v in predicted_topics:
        topicvector[trainrow,k]=v
    trainrow+=1

In [400]:
#valid dataset topic feature generation
validtopicvector = np.zeros((len(valid_dataframe),20))

In [401]:
validrow=0
for index, row in valid_dataframe.iterrows():
#     print (index, row['original'], '|', row['edit'])
    title_text = row['original'].replace('<', '').replace('/>', '')
    title_text = prepare_text_for_lda(title_text)
    title_doc_bow = dictionary.doc2bow(title_text)
    predicted_topics = ldamodel.get_document_topics(title_doc_bow)
    for k,v in predicted_topics:
        validtopicvector[validrow,k]=v
    validrow+=1

In [402]:
##test dataset topic feature generation
testtopicvector = np.zeros((len(test_dataframe),20))

In [403]:
testrow=0
for index, row in test_dataframe.iterrows():
#     print (index, row['original'], '|', row['edit'])
    title_text = row['original'].replace('<', '').replace('/>', '')
    title_text = prepare_text_for_lda(title_text)
    title_doc_bow = dictionary.doc2bow(title_text)
    predicted_topics = ldamodel.get_document_topics(title_doc_bow)
    for k,v in predicted_topics:
        testtopicvector[testrow,k]=v
    testrow+=1

In [404]:
topicfeature = pd.DataFrame(topicvector)

In [405]:
Validtopicfeature = pd.DataFrame(validtopicvector)

In [406]:
len(testtopicvector)

2413

In [407]:
testtopicfeature = pd.DataFrame(testtopicvector)

# TFIDF Features

In [408]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [409]:
def get_raw_text(df):
    corpus = []
    for index, row in df.iterrows():
        title = row['original'].replace('<', '').replace('/>', '')
        edit = row['edit']
        corpus.append( title + ' ' + edit )
    return corpus

In [410]:
train_corpus = get_raw_text(train_dataframe[["original","edit"]])
tfidfvectorizer = TfidfVectorizer(stop_words = "english",ngram_range=(1,2),min_df=0., max_df=1., use_idf=True).fit(train_corpus)

In [473]:
def separate_original_word_from_title(df):
    original_words = []
    contexts = []
    for index, row in df.iterrows():
        title = row['original']
        start_position = title.find('<')
        end_position = title.find('/>')
        original_words.append(title[start_position+1 : end_position])
        contexts.append(title[:start_position] + title[end_position+2 :])
    return original_words, contexts

def construct_tfidffeature_matrix(df, vectorizer):
    edit_words = df['edit'].tolist()
    X = vectorizer.transform(edit_words)
    return X

In [474]:
trainidf = construct_tfidffeature_matrix(train_dataframe,tfidfvectorizer)
valididf = construct_tfidffeature_matrix(valid_dataframe,tfidfvectorizer)
testidf = construct_tfidffeature_matrix(test_dataframe,tfidfvectorizer)

In [417]:
#trainidf = construct_tfidffeature_matrix(train_edit_corpus,tfidfvectorizer)
#valididf = construct_tfidffeature_matrix(valid_edit_corpus,tfidfvectorizer)
#testidf = construct_tfidffeature_matrix(test_edit_corpus,tfidfvectorizer)

In [418]:
trainidf.shape

(5067, 30656)

In [483]:
# Dimensionality reduction

svd = TruncatedSVD(n_components=2480, random_state=42)
trainX_svd = svd.fit_transform(trainidf)
print(f"Total variance explained: {np.sum(svd.explained_variance_ratio_):.2f}")
# The variance explained is quite low for real applications. We will investigate it later.

Total variance explained: 0.90


In [484]:
trainX_svd.shape

(5067, 2480)

In [485]:
validX_svd = svd.transform(valididf)

In [486]:
validX_svd.shape

(2172, 2480)

In [487]:
testX_svd = svd.transform(testidf)

In [488]:
testX_svd.shape

(2413, 2480)

# Feature Combination


In [ ]:
#trainX = pd.concat([pd.DataFrame(Train_X_pca),topicfeature],axis=1)

In [41]:
#trainX = pd.concat([pd.DataFrame(train_X_wordfeature),topicfeature],axis=1)

In [490]:
trainX = pd.concat([pd.DataFrame(train_X_wordfeature),topicfeature,pd.DataFrame(trainX_svd)],axis=1)

In [427]:
names=[]
for i in range(1,301):
    names.append("word_{}".format(i))
    #names.append("pca_{}".format(i))
for i in range(1,21):
    names.append("topic_{}".format(i))
trainX.columns = names

ValueError: Length mismatch: Expected axis has 3320 elements, new values have 320 elements

In [491]:
names=[]
for i in range(1,301):
    names.append("word_{}".format(i))
    #names.append("pca_{}".format(i))
for i in range(1,21):
    names.append("topic_{}".format(i))
for i in range(1,2481):
    names.append("tfidf_{}".format(i))
trainX.columns = names

In [492]:
trainX.head()

,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9,word_10,...,tfidf_2471,tfidf_2472,tfidf_2473,tfidf_2474,tfidf_2475,tfidf_2476,tfidf_2477,tfidf_2478,tfidf_2479,tfidf_2480
0,-0.072374,0.174289,0.037503,-0.022536,-0.073880,-0.000163,0.017714,-0.044800,-0.004150,0.044773,...,-1.965116e-17,1.320829e-16,4.391019e-18,1.975416e-16,-4.496729e-17,-7.708677e-17,-3.149607e-17,-1.135566e-16,-2.439455e-17,-1.231925e-16
1,0.022504,-0.057910,0.024001,0.128857,-0.047217,-0.074889,0.055676,-0.081177,0.220728,-0.019727,...,-1.745989e-17,-1.917894e-17,-6.228826e-17,-7.444996e-18,1.772628e-18,-4.990586e-18,3.639192e-17,1.955926e-17,7.325819e-17,-2.258457e-16
2,-0.022287,0.056885,-0.029835,0.109235,-0.090969,0.002720,0.011318,-0.053188,0.088619,0.053432,...,-1.647119e-17,3.547764e-18,4.940849e-17,3.896352e-19,-3.487722e-17,5.085586e-18,3.960303e-17,3.701195e-17,1.730891e-17,1.370648e-16
3,0.056274,-0.055262,0.143148,0.106059,-0.177531,-0.045736,0.000671,-0.087006,0.175863,0.118937,...,-7.751690e-18,1.783174e-17,2.133580e-18,-8.371598e-18,7.815590e-18,8.613051e-18,-8.367086e-18,-1.596698e-17,1.295218e-17,4.370400e-19
4,0.025635,0.059570,-0.039795,0.031372,-0.128052,0.072754,-0.006104,-0.038818,0.117767,0.112061,...,1.527607e-02,-5.515125e-02,4.298058e-02,-2.249188e-02,-3.977244e-02,-1.734172e-02,5.727280e-03,-1.842824e-02,9.720209e-03,1.003464e-03


In [493]:
Valid_X = pd.concat([pd.DataFrame(Valid_X_wordfeature),Validtopicfeature,pd.DataFrame(validX_svd)],axis=1)

In [494]:
Valid_X.columns=names

In [505]:
test_X = pd.concat([pd.DataFrame(test_X),testtopicfeature,pd.DataFrame(testX_svd)],axis=1)

In [506]:
test_X.columns=names

# Train Model

In [507]:
from sklearn.linear_model import ElasticNet,BayesianRidge,LinearRegression,Lasso
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.linear_model import Ridge
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from catboost import Pool

In [508]:
#rough model test
lasso_model = Lasso(alpha=1)
lr_model = LinearRegression()
etc_model = ElasticNet()
svr_model = SVR()
gbr_model = GradientBoostingRegressor()
ridge_model = Ridge(alpha=1)
xgb_model = XGBRegressor()
kernel_model = KernelRidge(alpha=1)
catmodel = CatBoostRegressor()

rmses ={}

model_dic = {"Lasso":lasso_model,
             "LinearRegression":lr_model,
             "ElasticNet":etc_model,
             "SVR":svr_model,
             "GBR":gbr_model,
             "Ridge":ridge_model,
             "Kernel":kernel_model,
             "XGB":xgb_model,
             "CAT":catmodel}
#model_names = ["BayesianRidge","LinearRegression","ElasticNet","SVR","GBR","Ridge"]
#model_dic = [br_model,lr_model,etc_model,svr_model,gbr_model,ridge_model]
for model_name,dicmodel in model_dic.items():
    trainmodel = dicmodel.fit(trainX,train_Y)
    valid_Y_hat = dicmodel.predict(Valid_X)
    rmse = np.sqrt(mean_squared_error(valid_Y, valid_Y_hat))
    rmses[model_name]=rmse

Learning rate set to 0.052909
0:	learn: 0.5850783	total: 219ms	remaining: 3m 38s
1:	learn: 0.5836311	total: 371ms	remaining: 3m 5s
2:	learn: 0.5823435	total: 524ms	remaining: 2m 54s
3:	learn: 0.5810310	total: 677ms	remaining: 2m 48s
4:	learn: 0.5798037	total: 829ms	remaining: 2m 44s
5:	learn: 0.5787787	total: 979ms	remaining: 2m 42s
6:	learn: 0.5774775	total: 1.13s	remaining: 2m 40s
7:	learn: 0.5761686	total: 1.28s	remaining: 2m 38s
8:	learn: 0.5750820	total: 1.45s	remaining: 2m 39s
9:	learn: 0.5740617	total: 1.6s	remaining: 2m 38s
10:	learn: 0.5729406	total: 1.75s	remaining: 2m 37s
11:	learn: 0.5719677	total: 1.91s	remaining: 2m 37s
12:	learn: 0.5709943	total: 2.06s	remaining: 2m 36s
13:	learn: 0.5700682	total: 2.21s	remaining: 2m 35s
14:	learn: 0.5691958	total: 2.36s	remaining: 2m 35s
15:	learn: 0.5682944	total: 2.51s	remaining: 2m 34s
16:	learn: 0.5675880	total: 2.67s	remaining: 2m 34s
17:	learn: 0.5669851	total: 2.81s	remaining: 2m 33s
18:	learn: 0.5664353	total: 2.96s	remaining: 2

158:	learn: 0.4864935	total: 24.4s	remaining: 2m 8s
159:	learn: 0.4860391	total: 24.5s	remaining: 2m 8s
160:	learn: 0.4856507	total: 24.7s	remaining: 2m 8s
161:	learn: 0.4851639	total: 24.8s	remaining: 2m 8s
162:	learn: 0.4847069	total: 25s	remaining: 2m 8s
163:	learn: 0.4842356	total: 25.1s	remaining: 2m 8s
164:	learn: 0.4838645	total: 25.3s	remaining: 2m 7s
165:	learn: 0.4834068	total: 25.4s	remaining: 2m 7s
166:	learn: 0.4828658	total: 25.6s	remaining: 2m 7s
167:	learn: 0.4821454	total: 25.7s	remaining: 2m 7s
168:	learn: 0.4815751	total: 25.9s	remaining: 2m 7s
169:	learn: 0.4812231	total: 26s	remaining: 2m 7s
170:	learn: 0.4807021	total: 26.2s	remaining: 2m 7s
171:	learn: 0.4801431	total: 26.3s	remaining: 2m 6s
172:	learn: 0.4797863	total: 26.5s	remaining: 2m 6s
173:	learn: 0.4793259	total: 26.7s	remaining: 2m 6s
174:	learn: 0.4788335	total: 26.8s	remaining: 2m 6s
175:	learn: 0.4781675	total: 27s	remaining: 2m 6s
176:	learn: 0.4774979	total: 27.1s	remaining: 2m 6s
177:	learn: 0.4767

316:	learn: 0.3976075	total: 48.5s	remaining: 1m 44s
317:	learn: 0.3970199	total: 48.7s	remaining: 1m 44s
318:	learn: 0.3965465	total: 48.8s	remaining: 1m 44s
319:	learn: 0.3961991	total: 49s	remaining: 1m 44s
320:	learn: 0.3955057	total: 49.1s	remaining: 1m 43s
321:	learn: 0.3949502	total: 49.3s	remaining: 1m 43s
322:	learn: 0.3944805	total: 49.4s	remaining: 1m 43s
323:	learn: 0.3940115	total: 49.6s	remaining: 1m 43s
324:	learn: 0.3935450	total: 49.7s	remaining: 1m 43s
325:	learn: 0.3928337	total: 49.9s	remaining: 1m 43s
326:	learn: 0.3921160	total: 50s	remaining: 1m 43s
327:	learn: 0.3917237	total: 50.2s	remaining: 1m 42s
328:	learn: 0.3912348	total: 50.4s	remaining: 1m 42s
329:	learn: 0.3907778	total: 50.5s	remaining: 1m 42s
330:	learn: 0.3902082	total: 50.7s	remaining: 1m 42s
331:	learn: 0.3897664	total: 50.8s	remaining: 1m 42s
332:	learn: 0.3892022	total: 51s	remaining: 1m 42s
333:	learn: 0.3887569	total: 51.1s	remaining: 1m 41s
334:	learn: 0.3883479	total: 51.3s	remaining: 1m 41s

472:	learn: 0.3296531	total: 1m 12s	remaining: 1m 20s
473:	learn: 0.3291978	total: 1m 12s	remaining: 1m 20s
474:	learn: 0.3289109	total: 1m 12s	remaining: 1m 20s
475:	learn: 0.3284687	total: 1m 12s	remaining: 1m 20s
476:	learn: 0.3279738	total: 1m 13s	remaining: 1m 20s
477:	learn: 0.3276225	total: 1m 13s	remaining: 1m 19s
478:	learn: 0.3272576	total: 1m 13s	remaining: 1m 19s
479:	learn: 0.3269473	total: 1m 13s	remaining: 1m 19s
480:	learn: 0.3266291	total: 1m 13s	remaining: 1m 19s
481:	learn: 0.3263311	total: 1m 13s	remaining: 1m 19s
482:	learn: 0.3259664	total: 1m 13s	remaining: 1m 19s
483:	learn: 0.3257506	total: 1m 14s	remaining: 1m 19s
484:	learn: 0.3254012	total: 1m 14s	remaining: 1m 18s
485:	learn: 0.3250314	total: 1m 14s	remaining: 1m 18s
486:	learn: 0.3246052	total: 1m 14s	remaining: 1m 18s
487:	learn: 0.3241851	total: 1m 14s	remaining: 1m 18s
488:	learn: 0.3238018	total: 1m 14s	remaining: 1m 18s
489:	learn: 0.3235343	total: 1m 15s	remaining: 1m 18s
490:	learn: 0.3232710	total:

626:	learn: 0.2780539	total: 1m 36s	remaining: 57.2s
627:	learn: 0.2775914	total: 1m 36s	remaining: 57s
628:	learn: 0.2773740	total: 1m 36s	remaining: 56.9s
629:	learn: 0.2770298	total: 1m 36s	remaining: 56.7s
630:	learn: 0.2768148	total: 1m 36s	remaining: 56.6s
631:	learn: 0.2764654	total: 1m 36s	remaining: 56.4s
632:	learn: 0.2760984	total: 1m 37s	remaining: 56.3s
633:	learn: 0.2757925	total: 1m 37s	remaining: 56.1s
634:	learn: 0.2754999	total: 1m 37s	remaining: 56s
635:	learn: 0.2751598	total: 1m 37s	remaining: 55.8s
636:	learn: 0.2748209	total: 1m 37s	remaining: 55.7s
637:	learn: 0.2743956	total: 1m 37s	remaining: 55.5s
638:	learn: 0.2741660	total: 1m 37s	remaining: 55.4s
639:	learn: 0.2739257	total: 1m 38s	remaining: 55.2s
640:	learn: 0.2735510	total: 1m 38s	remaining: 55.1s
641:	learn: 0.2732726	total: 1m 38s	remaining: 54.9s
642:	learn: 0.2729100	total: 1m 38s	remaining: 54.8s
643:	learn: 0.2725043	total: 1m 38s	remaining: 54.6s
644:	learn: 0.2723059	total: 1m 38s	remaining: 54.

783:	learn: 0.2357731	total: 2m	remaining: 33.2s
784:	learn: 0.2355986	total: 2m	remaining: 33s
785:	learn: 0.2352604	total: 2m	remaining: 32.9s
786:	learn: 0.2350644	total: 2m	remaining: 32.7s
787:	learn: 0.2347849	total: 2m 1s	remaining: 32.6s
788:	learn: 0.2345614	total: 2m 1s	remaining: 32.4s
789:	learn: 0.2343259	total: 2m 1s	remaining: 32.3s
790:	learn: 0.2341313	total: 2m 1s	remaining: 32.1s
791:	learn: 0.2339668	total: 2m 1s	remaining: 31.9s
792:	learn: 0.2337919	total: 2m 1s	remaining: 31.8s
793:	learn: 0.2335326	total: 2m 1s	remaining: 31.6s
794:	learn: 0.2331972	total: 2m 2s	remaining: 31.5s
795:	learn: 0.2329465	total: 2m 2s	remaining: 31.3s
796:	learn: 0.2327146	total: 2m 2s	remaining: 31.2s
797:	learn: 0.2324149	total: 2m 2s	remaining: 31s
798:	learn: 0.2321220	total: 2m 2s	remaining: 30.9s
799:	learn: 0.2318606	total: 2m 2s	remaining: 30.7s
800:	learn: 0.2315511	total: 2m 3s	remaining: 30.6s
801:	learn: 0.2312900	total: 2m 3s	remaining: 30.4s
802:	learn: 0.2309874	total:

941:	learn: 0.1993647	total: 2m 24s	remaining: 8.91s
942:	learn: 0.1992487	total: 2m 24s	remaining: 8.76s
943:	learn: 0.1991041	total: 2m 25s	remaining: 8.6s
944:	learn: 0.1988951	total: 2m 25s	remaining: 8.45s
945:	learn: 0.1987522	total: 2m 25s	remaining: 8.3s
946:	learn: 0.1985987	total: 2m 25s	remaining: 8.14s
947:	learn: 0.1983155	total: 2m 25s	remaining: 7.99s
948:	learn: 0.1980937	total: 2m 25s	remaining: 7.84s
949:	learn: 0.1979432	total: 2m 25s	remaining: 7.68s
950:	learn: 0.1976818	total: 2m 26s	remaining: 7.53s
951:	learn: 0.1975430	total: 2m 26s	remaining: 7.38s
952:	learn: 0.1974031	total: 2m 26s	remaining: 7.22s
953:	learn: 0.1972800	total: 2m 26s	remaining: 7.07s
954:	learn: 0.1970710	total: 2m 26s	remaining: 6.91s
955:	learn: 0.1968236	total: 2m 26s	remaining: 6.76s
956:	learn: 0.1966957	total: 2m 27s	remaining: 6.61s
957:	learn: 0.1964353	total: 2m 27s	remaining: 6.45s
958:	learn: 0.1963018	total: 2m 27s	remaining: 6.3s
959:	learn: 0.1960669	total: 2m 27s	remaining: 6.

In [509]:
for item,value in rmses.items():
    print (item,value)

Lasso 0.586252839148161
LinearRegression 0.6279094342018607
ElasticNet 0.586252839148161
SVR 0.5520305094125122
GBR 0.5741092229889524
Ridge 0.5737252233421949
Kernel 0.5733080279552392
XGB 0.5941612789674993
CAT 0.5624550654860999


In [450]:
import lightgbm

In [451]:
lgb = lightgbm.LGBMRegressor()
lgb.fit(trainX,train_Y)
valid_Y_hat = lgb.predict(Valid_X)
np.sqrt(mean_squared_error(valid_Y, valid_Y_hat))

0.5745028322950003

In [452]:
params = {
    'num_leaves': [10, 14,16],
    'learning_rate': [0.01, 0.03, 0.05],
    'max_depth': [4,5,7,10],
    'n_estimators': [400,500,700],
}

lgbgrid = GridSearchCV(lgb, params, scoring='neg_mean_squared_error', cv=5)
lgbgrid.fit(trainX, train_Y)
valid_Y_hat = lgbgrid.predict(Valid_X)
np.sqrt(mean_squared_error(valid_Y, valid_Y_hat))

KeyboardInterrupt: 

In [ ]:
lgbgrid = lightgbm.LGBMRegressor(learning_rate=0.1, max_depth=15, n_estimators=800, num_leaves=25)
lgbgrid.fit(trainX,train_Y)
valid_Y_hat = lgbgrid.predict(Valid_X)
np.sqrt(mean_squared_error(valid_Y, valid_Y_hat))

In [347]:
lgbgrid.best_estimator_

LGBMRegressor(learning_rate=0.03, max_depth=10, n_estimators=400, num_leaves=10)

In [ ]:
LGBMRegressor(learning_rate=0.03, max_depth=5, n_estimators=500, num_leaves=14)
LGBMRegressor(learning_rate=0.03, max_depth=10, n_estimators=400, num_leaves=10)
LGBMRegressor(learning_rate=0.03, max_depth=5, n_estimators=400, num_leaves=15)

In [362]:
train_Y_hat = lgbgrid.predict(trainX)

np.sqrt(mean_squared_error(train_Y, train_Y_hat))

0.34391335330904654

In [317]:
train_dataset = Pool(trainX, train_Y) 

catmodel_grid = CatBoostRegressor(loss_function='RMSE')
parameters ={'iterations': [200],
        'learning_rate': [0.15,0.5,1,5],
        'depth': [4,10],
        'l2_leaf_reg': [5,20,30]}
catmodel_grid.grid_search(parameters, train_dataset)


0:	learn: 0.9866666	test: 1.0219747	best: 1.0219747 (0)	total: 73.4ms	remaining: 14.6s
1:	learn: 0.8930426	test: 0.9295099	best: 0.9295099 (1)	total: 121ms	remaining: 12s
2:	learn: 0.8181703	test: 0.8557614	best: 0.8557614 (2)	total: 168ms	remaining: 11s
3:	learn: 0.7586653	test: 0.7969990	best: 0.7969990 (3)	total: 216ms	remaining: 10.6s
4:	learn: 0.7126997	test: 0.7510505	best: 0.7510505 (4)	total: 265ms	remaining: 10.3s
5:	learn: 0.6768741	test: 0.7152547	best: 0.7152547 (5)	total: 321ms	remaining: 10.4s
6:	learn: 0.6493262	test: 0.6873313	best: 0.6873313 (6)	total: 371ms	remaining: 10.2s
7:	learn: 0.6287356	test: 0.6660050	best: 0.6660050 (7)	total: 419ms	remaining: 10s
8:	learn: 0.6132940	test: 0.6497550	best: 0.6497550 (8)	total: 467ms	remaining: 9.91s
9:	learn: 0.6016752	test: 0.6373920	best: 0.6373920 (9)	total: 516ms	remaining: 9.79s
10:	learn: 0.5927605	test: 0.6282255	best: 0.6282255 (10)	total: 570ms	remaining: 9.79s
11:	learn: 0.5862798	test: 0.6209804	best: 0.6209804 (11)

98:	learn: 0.4765889	test: 0.5814566	best: 0.5814566 (98)	total: 5.04s	remaining: 5.14s
99:	learn: 0.4755637	test: 0.5815515	best: 0.5814566 (98)	total: 5.09s	remaining: 5.09s
100:	learn: 0.4743385	test: 0.5811557	best: 0.5811557 (100)	total: 5.14s	remaining: 5.04s
101:	learn: 0.4730394	test: 0.5814408	best: 0.5811557 (100)	total: 5.19s	remaining: 4.99s
102:	learn: 0.4718687	test: 0.5812493	best: 0.5811557 (100)	total: 5.24s	remaining: 4.94s
103:	learn: 0.4707385	test: 0.5816433	best: 0.5811557 (100)	total: 5.3s	remaining: 4.89s
104:	learn: 0.4695398	test: 0.5811606	best: 0.5811557 (100)	total: 5.35s	remaining: 4.84s
105:	learn: 0.4680975	test: 0.5812954	best: 0.5811557 (100)	total: 5.4s	remaining: 4.79s
106:	learn: 0.4668370	test: 0.5809620	best: 0.5809620 (106)	total: 5.45s	remaining: 4.73s
107:	learn: 0.4659018	test: 0.5807685	best: 0.5807685 (107)	total: 5.5s	remaining: 4.69s
108:	learn: 0.4650827	test: 0.5807837	best: 0.5807685 (107)	total: 5.56s	remaining: 4.64s
109:	learn: 0.464

194:	learn: 0.3869595	test: 0.5763958	best: 0.5761550 (179)	total: 9.95s	remaining: 255ms
195:	learn: 0.3862562	test: 0.5763244	best: 0.5761550 (179)	total: 10s	remaining: 204ms
196:	learn: 0.3854358	test: 0.5766613	best: 0.5761550 (179)	total: 10s	remaining: 153ms
197:	learn: 0.3844985	test: 0.5763722	best: 0.5761550 (179)	total: 10.1s	remaining: 102ms
198:	learn: 0.3839297	test: 0.5764162	best: 0.5761550 (179)	total: 10.2s	remaining: 51ms
199:	learn: 0.3833035	test: 0.5766087	best: 0.5761550 (179)	total: 10.2s	remaining: 0us

bestTest = 0.5761549731
bestIteration = 179

0:	loss: 0.5761550	best: 0.5761550 (0)	total: 10.5s	remaining: 4m 1s
0:	learn: 0.7479617	test: 0.7854801	best: 0.7854801 (0)	total: 94.2ms	remaining: 18.7s
1:	learn: 0.6232121	test: 0.6606094	best: 0.6606094 (1)	total: 143ms	remaining: 14.2s
2:	learn: 0.5856323	test: 0.6219386	best: 0.6219386 (2)	total: 195ms	remaining: 12.8s
3:	learn: 0.5724096	test: 0.6069676	best: 0.6069676 (3)	total: 246ms	remaining: 12s
4:	learn:

87:	learn: 0.3377708	test: 0.6150403	best: 0.5943548 (22)	total: 4.61s	remaining: 5.87s
88:	learn: 0.3364520	test: 0.6155336	best: 0.5943548 (22)	total: 4.66s	remaining: 5.82s
89:	learn: 0.3351788	test: 0.6148660	best: 0.5943548 (22)	total: 4.71s	remaining: 5.76s
90:	learn: 0.3332452	test: 0.6147104	best: 0.5943548 (22)	total: 4.76s	remaining: 5.71s
91:	learn: 0.3307991	test: 0.6152727	best: 0.5943548 (22)	total: 4.82s	remaining: 5.65s
92:	learn: 0.3286190	test: 0.6147565	best: 0.5943548 (22)	total: 4.87s	remaining: 5.6s
93:	learn: 0.3270988	test: 0.6141567	best: 0.5943548 (22)	total: 4.92s	remaining: 5.55s
94:	learn: 0.3251799	test: 0.6138294	best: 0.5943548 (22)	total: 4.97s	remaining: 5.49s
95:	learn: 0.3240395	test: 0.6135233	best: 0.5943548 (22)	total: 5.02s	remaining: 5.44s
96:	learn: 0.3223869	test: 0.6138654	best: 0.5943548 (22)	total: 5.07s	remaining: 5.39s
97:	learn: 0.3204028	test: 0.6139033	best: 0.5943548 (22)	total: 5.12s	remaining: 5.33s
98:	learn: 0.3189193	test: 0.6134

180:	learn: 0.2093709	test: 0.6242642	best: 0.5943548 (22)	total: 9.44s	remaining: 991ms
181:	learn: 0.2084546	test: 0.6243352	best: 0.5943548 (22)	total: 9.49s	remaining: 939ms
182:	learn: 0.2078277	test: 0.6240424	best: 0.5943548 (22)	total: 9.54s	remaining: 886ms
183:	learn: 0.2069818	test: 0.6242941	best: 0.5943548 (22)	total: 9.59s	remaining: 834ms
184:	learn: 0.2058123	test: 0.6244555	best: 0.5943548 (22)	total: 9.64s	remaining: 782ms
185:	learn: 0.2049140	test: 0.6244117	best: 0.5943548 (22)	total: 9.7s	remaining: 730ms
186:	learn: 0.2041453	test: 0.6247017	best: 0.5943548 (22)	total: 9.74s	remaining: 678ms
187:	learn: 0.2033800	test: 0.6241651	best: 0.5943548 (22)	total: 9.79s	remaining: 625ms
188:	learn: 0.2024708	test: 0.6246487	best: 0.5943548 (22)	total: 9.85s	remaining: 573ms
189:	learn: 0.2012253	test: 0.6245909	best: 0.5943548 (22)	total: 9.9s	remaining: 521ms
190:	learn: 0.2001365	test: 0.6245714	best: 0.5943548 (22)	total: 9.95s	remaining: 469ms
191:	learn: 0.1991033	t

...

37:	learn: 0.5476455	test: 0.5650779	best: 0.5650779 (37)	total: 2.1s	remaining: 8.96s
38:	learn: 0.5467699	test: 0.5645155	best: 0.5645155 (38)	total: 2.16s	remaining: 8.92s
39:	learn: 0.5457338	test: 0.5646408	best: 0.5645155 (38)	total: 2.21s	remaining: 8.86s
40:	learn: 0.5448350	test: 0.5646552	best: 0.5645155 (38)	total: 2.27s	remaining: 8.79s
41:	learn: 0.5437209	test: 0.5645309	best: 0.5645155 (38)	total: 2.32s	remaining: 8.74s
42:	learn: 0.5428535	test: 0.5643748	best: 0.5643748 (42)	total: 2.38s	remaining: 8.68s
43:	learn: 0.5410777	test: 0.5640969	best: 0.5640969 (43)	total: 2.43s	remaining: 8.62s
44:	learn: 0.5396790	test: 0.5638261	best: 0.5638261 (44)	total: 2.49s	remaining: 8.56s
45:	learn: 0.5386369	test: 0.5639476	best: 0.5638261 (44)	total: 2.54s	remaining: 8.5s
46:	learn: 0.5370687	test: 0.5637517	best: 0.5637517 (46)	total: 2.6s	remaining: 8.45s
47:	learn: 0.5356208	test: 0.5638424	best: 0.5637517 (46)	total: 2.65s	remaining: 8.4s
48:	learn: 0.5347531	test: 0.5639754

135:	learn: 0.4242597	test: 0.5666458	best: 0.5629113 (63)	total: 7.4s	remaining: 3.48s
136:	learn: 0.4230893	test: 0.5665922	best: 0.5629113 (63)	total: 7.46s	remaining: 3.43s
137:	learn: 0.4222729	test: 0.5664451	best: 0.5629113 (63)	total: 7.51s	remaining: 3.37s
138:	learn: 0.4210580	test: 0.5669994	best: 0.5629113 (63)	total: 7.56s	remaining: 3.32s
139:	learn: 0.4202181	test: 0.5667155	best: 0.5629113 (63)	total: 7.62s	remaining: 3.26s
140:	learn: 0.4190938	test: 0.5666953	best: 0.5629113 (63)	total: 7.67s	remaining: 3.21s
141:	learn: 0.4182348	test: 0.5667207	best: 0.5629113 (63)	total: 7.72s	remaining: 3.15s
142:	learn: 0.4174023	test: 0.5665670	best: 0.5629113 (63)	total: 7.78s	remaining: 3.1s
143:	learn: 0.4159879	test: 0.5668643	best: 0.5629113 (63)	total: 7.83s	remaining: 3.04s
144:	learn: 0.4151678	test: 0.5670993	best: 0.5629113 (63)	total: 7.88s	remaining: 2.99s
145:	learn: 0.4139198	test: 0.5673558	best: 0.5629113 (63)	total: 7.94s	remaining: 2.94s
146:	learn: 0.4128506	t

30:	learn: 0.5494752	test: 0.5814664	best: 0.5814664 (30)	total: 1.71s	remaining: 9.31s
31:	learn: 0.5480858	test: 0.5814617	best: 0.5814617 (31)	total: 1.76s	remaining: 9.25s
32:	learn: 0.5468672	test: 0.5809004	best: 0.5809004 (32)	total: 1.82s	remaining: 9.2s
33:	learn: 0.5456978	test: 0.5807892	best: 0.5807892 (33)	total: 1.87s	remaining: 9.13s
34:	learn: 0.5447400	test: 0.5804031	best: 0.5804031 (34)	total: 1.92s	remaining: 9.07s
35:	learn: 0.5436485	test: 0.5802489	best: 0.5802489 (35)	total: 1.98s	remaining: 9.01s
36:	learn: 0.5426586	test: 0.5803213	best: 0.5802489 (35)	total: 2.03s	remaining: 8.95s
37:	learn: 0.5412826	test: 0.5803567	best: 0.5802489 (35)	total: 2.09s	remaining: 8.9s
38:	learn: 0.5402207	test: 0.5802416	best: 0.5802416 (38)	total: 2.14s	remaining: 8.85s
39:	learn: 0.5393674	test: 0.5799186	best: 0.5799186 (39)	total: 2.2s	remaining: 8.79s
40:	learn: 0.5382713	test: 0.5799219	best: 0.5799186 (39)	total: 2.25s	remaining: 8.73s
41:	learn: 0.5372178	test: 0.580283

126:	learn: 0.4296511	test: 0.5786816	best: 0.5777798 (67)	total: 6.9s	remaining: 3.97s
127:	learn: 0.4281944	test: 0.5787165	best: 0.5777798 (67)	total: 6.95s	remaining: 3.91s
128:	learn: 0.4271230	test: 0.5788738	best: 0.5777798 (67)	total: 7.01s	remaining: 3.86s
129:	learn: 0.4263391	test: 0.5792717	best: 0.5777798 (67)	total: 7.06s	remaining: 3.8s
130:	learn: 0.4254496	test: 0.5792203	best: 0.5777798 (67)	total: 7.11s	remaining: 3.74s
131:	learn: 0.4247290	test: 0.5791877	best: 0.5777798 (67)	total: 7.16s	remaining: 3.69s
132:	learn: 0.4236163	test: 0.5794057	best: 0.5777798 (67)	total: 7.22s	remaining: 3.63s
133:	learn: 0.4225220	test: 0.5791830	best: 0.5777798 (67)	total: 7.27s	remaining: 3.58s
134:	learn: 0.4216660	test: 0.5794047	best: 0.5777798 (67)	total: 7.32s	remaining: 3.52s
135:	learn: 0.4207255	test: 0.5791751	best: 0.5777798 (67)	total: 7.37s	remaining: 3.47s
136:	learn: 0.4194510	test: 0.5792394	best: 0.5777798 (67)	total: 7.43s	remaining: 3.42s
137:	learn: 0.4186170	t

{'params': {'depth': 4,
  'l2_leaf_reg': 5,
  'iterations': 200,
  'learning_rate': 0.15},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45

In [324]:
valid_Y_hat = catmodel.predict(Valid_X)

np.sqrt(mean_squared_error(valid_Y, valid_Y_hat))

0.5631653706862305

In [325]:
train_Y_hat = catmodel.predict(trainX)

np.sqrt(mean_squared_error(train_Y, train_Y_hat))

0.18856167001611557

In [466]:
#Try Ridge regression
ridgemodel = Ridge()
parameters = {'alpha':[8,6,9]}
Ridge_reg= GridSearchCV(ridgemodel, parameters, scoring='neg_mean_squared_error',cv=5)
Ridge_reg.fit(trainX,train_Y)

GridSearchCV(cv=5, estimator=Ridge(), param_grid={'alpha': [8, 6, 9]},
             scoring='neg_mean_squared_error')

In [467]:
Ridge_reg.best_estimator_

Ridge(alpha=8)

In [468]:
valid_Y_hat = Ridge_reg.predict(Valid_X)

np.sqrt(mean_squared_error(valid_Y, valid_Y_hat))

0.5547666206507679

In [469]:
train_Y_hat = Ridge_reg.predict(trainX)

np.sqrt(mean_squared_error(train_Y, train_Y_hat))

0.48930393430808977

In [288]:
#Try Gradient Boosting Regression

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold
gbr= GradientBoostingRegressor()
parameters = {'learning_rate': [0.02,0.005],
                  'subsample'    : [0.9],
                  'n_estimators' : [100],
                  'max_depth'    : [8,15,30]
                 }
gbr_grid = GridSearchCV(gbr,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

gbr_grid.fit(trainX,
         train_Y)

gbr_grid.best_params_

Fitting 2 folds for each of 27 candidates, totalling 54 fits


{'learning_rate': 0.02, 'max_depth': 8, 'n_estimators': 100, 'subsample': 0.9}

In [ ]:
{'learning_rate': 0.02, 'max_depth': 8, 'n_estimators': 100, 'subsample': 0.9}

In [290]:
valid_Y_hat = gbr_grid.predict(Valid_X)

np.sqrt(mean_squared_error(valid_Y, valid_Y_hat))

0.5712989548526111

In [289]:
train_Y_hat = gbr_grid.predict(trainX)

np.sqrt(mean_squared_error(train_Y, train_Y_hat))

0.3690459862280796

In [471]:
paramsm={'learning_rate': 0.03, 'max_depth': 4, 'n_estimators': 100, 'subsample': 0.5}
gbr = GradientBoostingRegressor(random_state=2, **paramsm)

# Fit the classifier to the training set
gbr.fit(trainX, train_Y)

valid_Y_hat = gbr.predict(Valid_X)

np.sqrt(mean_squared_error(valid_Y, valid_Y_hat))

0.5689799354522234

In [472]:
train_Y_hat = gbr.predict(trainX)

np.sqrt(mean_squared_error(train_Y, train_Y_hat))

0.4975055632714873

In [510]:
train_Y_hat = svr_model.predict(trainX)

np.sqrt(mean_squared_error(train_Y, train_Y_hat))

0.24147290704461954

In [251]:
valid_Y_hat = svr_model.predict(Valid_X)

np.sqrt(mean_squared_error(valid_Y, valid_Y_hat))

0.5559558514213871

In [274]:
svr_model.gamma

'scale'

In [444]:
kr = GridSearchCV(KernelRidge(kernel='rbf', gamma=0.1),
                  param_grid={"alpha": [1,1.2,0.8],
                              "gamma": np.logspace(-2, 2, 5)})
kr.fit(trainX,train_Y)
kr.best_params_

{'alpha': 1, 'gamma': 0.1}

In [445]:
valid_Y_hat = kr.predict(Valid_X)
np.sqrt(mean_squared_error(valid_Y, valid_Y_hat))

0.5537546706961307

In [446]:
train_Y_hat = kr.predict(trainX)

np.sqrt(mean_squared_error(train_Y, train_Y_hat))

0.45642478270915754

In [513]:
parameters = {'kernel': ['rbf'], 'gamma': [0.05,0.15,0.4,0.9],'C': [0.9],'epsilon': [0.2]}
svr = SVR()
SVR_gridcv = GridSearchCV(svr, parameters,cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
SVR_gridcv.fit(trainX, train_Y)

SVR_gridcv.best_params_

{'C': 0.9, 'epsilon': 0.2, 'gamma': 0.4, 'kernel': 'rbf'}

In [ ]:
{'C': 0.4, 'epsilon': 0.15, 'gamma': 0.15, 'kernel': 'rbf'}
{'C': 0.45, 'epsilon': 0.2, 'gamma': 0.2, 'kernel': 'rbf'}
{'C': 0.8, 'epsilon': 0.2, 'gamma': 0.5, 'kernel': 'rbf'}
{'C': 0.9, 'epsilon': 0.2, 'gamma': 0.5, 'kernel': 'rbf'}
{'C': 0.9, 'epsilon': 0.2, 'gamma': 0.45, 'kernel': 'rbf'}
{'C': 0.9, 'epsilon': 0.2, 'gamma': 0.4, 'kernel': 'rbf'}

In [522]:
valid_Y_hat = SVR_gridcv.predict(Valid_X)
np.sqrt(mean_squared_error(valid_Y, valid_Y_hat))

0.5465062443310639

In [515]:
train_Y_hat = SVR_gridcv.predict(trainX)

np.sqrt(mean_squared_error(train_Y, train_Y_hat))

0.2449626983579028

In [284]:
xgb1 = xgb.XGBRegressor()
parameters = {'nthread':[4], 
              'objective':['reg:squarederror'],
              'learning_rate': [.03, 0.05, .07], 
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [250]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(trainX, train_Y)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[02:19:43] WARNING: D:\bld\xgboost-split_1631904903843\work\src\learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




GridSearchCV(cv=2,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=...
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_parameters=None,
                                    verbosity=None),
             n_

In [287]:
xgb_grid.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.03, max_delta_step=0, max_depth=5,
             min_child_weight=4, missing=nan, monotone_constraints='()',
             n_estimators=250, n_jobs=4, nthread=4, num_parallel_tree=1,
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             silent=1, subsample=0.7, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [285]:
valid_Y_hat = xgb_grid.predict(Valid_X)
np.sqrt(mean_squared_error(valid_Y, valid_Y_hat))

0.5600658260896076

In [286]:
train_Y_hat = xgb_grid.predict(trainX) 
np.sqrt(mean_squared_error(train_Y, train_Y_hat))

0.3443682091909952

In [94]:
def write_test_prediction(df, pred, filepath):
    with open(filepath, 'w') as outfile:
        outfile.write('{},{}\n'.format('id', 'pred'))
        for index, row in df.iterrows():
            outfile.write('{},{}\n'.format(row['id'], pred[index]))

In [123]:
test_Y_hat = Ridge_reg.predict(test_X)
write_test_prediction(test_dataframe, test_Y_hat, './ridge-regression_editword2vec_topicembedding_tfidf_1011.csv')

In [ ]:
test_Y_hat = gbr.predict(test_X)
write_test_prediction(test_dataframe, test_Y_hat, './gbr-regression_edit_pca_topicembedding1010.csv')

In [516]:
test_Y_hat = SVR_gridcv.predict(test_X)
write_test_prediction(test_dataframe, test_Y_hat, './svr_editword2vec_topicembedding_tfidf_1013.csv')

In [104]:
test_Y_hat = kr.predict(test_X)
write_test_prediction(test_dataframe, test_Y_hat, './kr_editword2vec_topicembedding_tfidf_1011.csv')

In [110]:
test_Y_hat = catmodel.predict(test_X)
write_test_prediction(test_dataframe, test_Y_hat, './cat_editword2vec_topicembedding_tfidf_1012.csv')